##**Pydantic is the most widely used data validation library for Python.**

##**Pydantic has many advantages, such as:**

##**Type safety:** Pydantic enforces type hints at runtime, ensuring that your data conforms to the expected types and formats. It also supports custom data types, such as enums, UUIDs, IP addresses, etc.
##**User-friendly errors:** Pydantic provides informative and readable error messages when validation fails, including the location, type, and input of the error. It also provides links to the documentation for each error type.
##**Performance:** Pydantic’s core validation logic is written in Rust, making it one of the fastest data validation libraries for Python. It also supports lazy validation and caching for improved efficiency.
##**Ease of use:** Pydantic is simple and intuitive to use, requiring minimal boilerplate code and configuration. It works well with many popular IDEs and static analysis tools, such as PyCharm, VS Code, mypy, etc.

###pip install pydantic

###"pedantic" refer's to the library's meticulous approach to **data validation** and **type enforcement.**

In [2]:
from datetime import datetime
from typing import Tuple

from pydantic import BaseModel


class Delivery(BaseModel):
    timestamp: datetime
    dimensions: Tuple[int, int]


m = Delivery(timestamp='2020-01-02T03:04:05Z', dimensions=['10', '20'])
print(repr(m.timestamp))
#> datetime.datetime(2020, 1, 2, 3, 4, 5, tzinfo=TzInfo(UTC))
print(m.dimensions)
#> (10, 20)

datetime.datetime(2020, 1, 2, 3, 4, 5, tzinfo=TzInfo(UTC))
(10, 20)


###**Creating a custom class that inherits from BaseModel:**

In [3]:
from datetime import datetime

from pydantic import BaseModel, PositiveInt


class User(BaseModel):
    id: int
    name: str = 'John Doe'
    signup_ts: datetime | None
    tastes: dict[str, PositiveInt]


external_data = {
    'id': 123,
    'signup_ts': '2019-06-01 12:22',
    'tastes': {
        'wine': 9,
        b'cheese': 7,
        'cabbage': '1',
    },
}

user = User(**external_data)

print(user.id)
#> 123
print(user.model_dump())
"""
{
    'id': 123,
    'name': 'John Doe',
    'signup_ts': datetime.datetime(2019, 6, 1, 12, 22),
    'tastes': {'wine': 9, 'cheese': 7, 'cabbage': 1},
}
"""

123
{'id': 123, 'name': 'John Doe', 'signup_ts': datetime.datetime(2019, 6, 1, 12, 22), 'tastes': {'wine': 9, 'cheese': 7, 'cabbage': 1}}


"\n{\n    'id': 123,\n    'name': 'John Doe',\n    'signup_ts': datetime.datetime(2019, 6, 1, 12, 22),\n    'tastes': {'wine': 9, 'cheese': 7, 'cabbage': 1},\n}\n"

###**If validation fails, Pydantic will raise an error with a breakdown of what was wrong:**



In [8]:
from datetime import datetime
from pydantic import ValidationError

from pydantic import BaseModel, PositiveInt


class User(BaseModel):
    id: int
    name: str = 'samim'
    signup_ts: datetime | None
    tastes: dict[str, PositiveInt]

external_data = {
    "id": 0,
    "name": "lisa",
    "signup_ts": "2019-06-01 12:22",
    "tastes": {
        "fish": "9",
        "tomato": "7",
        "cabbage": "1"
    }
}
user = User(**external_data)
print(user.id)

0


In [9]:
from datetime import datetime
from pydantic import ValidationError

from pydantic import BaseModel, PositiveInt


class User(BaseModel):
    id: int
    name: str = 'samim'
    signup_ts: datetime | None
    tastes: dict[str, PositiveInt]

external_data = {'id': 'not an int', 'tastes': {}}

try:
  user = User(**external_data)
  print(user.id)
except ValidationError as e:
  print(e.errors())


[{'type': 'int_parsing', 'loc': ('id',), 'msg': 'Input should be a valid integer, unable to parse string as an integer', 'input': 'not an int', 'url': 'https://errors.pydantic.dev/2.7/v/int_parsing'}, {'type': 'missing', 'loc': ('signup_ts',), 'msg': 'Field required', 'input': {'id': 'not an int', 'tastes': {}}, 'url': 'https://errors.pydantic.dev/2.7/v/missing'}]


###The main way to use Pydantic is to **create custom classes** that **inherit from BaseModel,** which is the base class for all Pydantic models. You can then define the attributes of your model using type annotations, and optionally provide default values or validators.

###For example, let’s create a simple model for a user:

In [10]:
from datetime import datetime
from typing import List, Optional
from pydantic import BaseModel

class User(BaseModel):
    id: int
    name: str = 'John Doe'
    signup_ts: Optional[datetime] = None
    friends: List[int] = []

###This model defines four attributes: id, name, signup_ts, and friends. The id attribute is required and must be an integer. The name attribute has a default value of 'John Doe' and must be a string. The signup_ts attribute is optional and can be either a datetime object or None. The friends attribute has an empty list as the default value and must be a list of integers.

###To **create an instance** of this model, you can **pass a dictionary of values** to the constructor:

In [11]:
external_data = {
    'id': 123,
    'signup_ts': '2019-06-01 12:22',
    'friends': [1, '2', b'3'],
}

user = User(**external_data)

###Pydantic will **automatically validate and parse** the input data according to the type annotations. It will also coerce the data to the correct type where appropriate. For example, it will convert the string '2019-06-01 12:22' to a datetime object, and the bytes b'3' to an integer.

###You can access the attributes of the model as normal:

In [12]:
print(user.id) # 123
print(user.name) # John Doe
print(user.signup_ts) # 2019-06-01 12:22:00
print(user.friends) # [1, 2, 3]

123
John Doe
2019-06-01 12:22:00
[1, 2, 3]


###Or as a JSON string using the json() method:

In [13]:
print(user.json())
# {"id": 123, "name": "John Doe", "signup_ts": "2019-06-01T12:22:00", "friends": [1, 2, 3]}

{"id":123,"name":"John Doe","signup_ts":"2019-06-01T12:22:00","friends":[1,2,3]}


###If the input data is invalid or missing some required values, Pydantic will **raise a ValidationError** with a **detailed breakdown** of what went wrong:

In [14]:
external_data = {
    'id': 'not an int',
    'tastes': {},
}

try:
    user = User(**external_data)
except ValidationError as e:
    print(e.errors())
# [
#     {
#         'type': 'int_parsing',
#         'loc': ('id',),
#         'msg': 'Input should be a valid integer, unable to parse string as an integer',
#         'input': 'not an int',
#         'url': 'https://errors.pydantic.dev/2/v/int_parsing',
#     },
#     {
#         'type': 'missing',
#         'loc': ('signup_ts',),
#         'msg': 'Field required',
#         'input': {'id': 'not an int', 'tastes': {}},
#         'url': 'https://errors.pydantic.dev/2/v/missing',
#     },
# ]

[{'type': 'int_parsing', 'loc': ('id',), 'msg': 'Input should be a valid integer, unable to parse string as an integer', 'input': 'not an int', 'url': 'https://errors.pydantic.dev/2.7/v/int_parsing'}]
